# Preprocessamento do DODF para segmentacao 

Separa sentencas e as rotula no padrao IOB (B=sentenca de inicio do ato, I=sentenca continuacao do ato, O=sentenca nao faz parte do ato)

In [1]:
import json
import pandas as pd
import glob
import re

## Parte 1 - Identificando os DODFs com anotacao de aposentadoria

In [2]:
df = pd.read_csv("labeled.csv")
df.columns

Index(['DATA_DODF', 'NUM_DODF', 'ATO', 'EMPRESA_ATO', 'COD_MATRICULA_ATO',
       'NOME_ATO', 'CARGO', 'CLASSE', 'PADRAO', 'QUADRO', 'PROCESSO',
       'FUND_LEGAL', 'text', 'labels'],
      dtype='object')

In [3]:
month = ["01_Janeiro/", "02_Fevereiro/", "03_Março/", "04_Abril/", "05_Maio/", "06_Junho/", "07_Julho/", "08_Agosto/", "09_Setembro/", "10_Outubro/", "11_Novembro/", "12_Dezembro/"]
dodfs = []
dic = {}

for i in range(len(df)):
    num_dodf = int(df.loc[i, 'NUM_DODF'])
    data_dodf= df.loc[i, 'DATA_DODF']
    if df.loc[i, 'text']:
        if (num_dodf, data_dodf) not in dic:
            dic[(num_dodf, data_dodf)] = -1
            dia, mes, ano = data_dodf.split('/')
            path = "../dodfs/results/json/" + ano + "/" + month[int(mes)-1]
            for i in glob.glob(path+'*.json'):
                aux = i.split('/')[-1]
                aux = aux.split()
                if int(aux[1])==num_dodf and aux[2] == data_dodf.replace('/', '-') and not re.search("SUPLEMENTO", ' '.join(aux)):
                    dic[(num_dodf, data_dodf)] = i
                    dodfs.append(i)

In [4]:
len(dodfs), len(dic)

(226, 227)

In [5]:
for i in dic:
    if dic[i] not in dodfs:
        print("Missing DODF:", i)

Missing DODF: (76, '20/04/2018')


In [6]:
dodfs.append("../dodfs/results/json/2018/04_Abril/DODF 076 20-04-2018 SUPLEMENTO.json")

## Parte 2 - Anotando os documentos no padrao IOB

In [7]:
path = "../dodfs/results/json/2018/04_Abril/DODF 019 04-04-2018 EDICAO EXTRA.json"
f = open(path)
data = json.load(f)

In [8]:
for i in data:
    print(i[4].replace('\n', ' '))
    print("-----------------------------------------------------------------------------------------------")

Este documento pode ser verificado no endereco eletronico http://www.in.gov.br/autenticidade.html , pelo codigo 50102018040400001 Documento assinado digitalmente conforme MP n 2.200-2 de 24/08/2001, que institui a Infraestrutura de Chaves Publicas Brasileira - ICP-Brasil.
-----------------------------------------------------------------------------------------------
ANO XLVII EDICAO EXTRA No- 19 BRASILIA - DF, QUARTA-FEIRA, 4 DE ABRIL DE 2018
-----------------------------------------------------------------------------------------------
SUMARIO SECAO I PAG. SECAO II PAG. SECAO III PAG.
-----------------------------------------------------------------------------------------------
Secretaria de Estado de Saude ......................................................... . 2 8
-----------------------------------------------------------------------------------------------
Secretaria de Estado de Educacao ................................................... . 1 6
------------------------------

In [46]:
full_base = []
not_found = []
cnt = 0
temp = -1
for dodf in dodfs:
    f = open(dodf)
    data = json.load(f)
    aux = df.loc[(df['NUM_DODF'] == int(dodf.split('/')[-1].split()[1])) & (df['DATA_DODF'] == dodf.split('/')[-1].split()[2].replace('-', '/'))]
    for j in range(len(data)):
        data[j].append([])
    for i in aux.index:
        cnt += 1
        if not df.loc[i, 'text'] or pd.isna(df.loc[i, 'text']):
            continue
        find = False
        for j in range(len(data)):
            entity = re.search(df.loc[i, 'text'].replace('(', '\(').replace(')', '\)'), data[j][4].replace('\n', ' '))
            if entity:
                temp = entity
                find = True
                data[j][5].append(entity.span())
        if not find:
            not_found.append((dodf, df.loc[i, 'text'], i))
    full_base.append(data)


In [47]:
len(not_found), cnt

(111, 5516)

In [41]:
a = "OLA MUNDO"
e = re.search("MUN", a)
e.span

<function Match.span(group=0, /)>

In [21]:
for i in not_found:
    print(i[0])
    print(i[1])
    print("Index:", i[2])
    print('\n\n')

../dodfs/results/json/2019/01_Janeiro/DODF 248 01-01-2019 INTEGRA.json
CONCEDER, aposentadoria voluntaria integral, a servidora MARIA FRASSINETTE DE FREITAS VASCONCELOS, matricula n.o 82.943-9, no cargo de Agente de Gestao de Residuos Solidos, Classe Unica, Padrao X, do Quadro de Pessoal do Servico de Limpeza Urbana do Distrito Federal, nos termos do artigo 3o, incisos I, II e III, e Paragrafo unico da Emenda Constitucional n.o 47 de 05/07/2005, combinado com o artigo 44 da Lei Complementar n.o 769, de 30/06/2008. Lotacao: Servico de Limpeza Urbana do Distrito Federal. Processo SEI n.o 00094-00012948/2018-15
Index: 15



../dodfs/results/json/2019/01_Janeiro/DODF 248 01-01-2019 INTEGRA.json
APOSENTAR, a servidora CLAUDIA HELENA DA SILVA SANTOS, matricula n.o 32.219-9, cargo de Tecnico em Politicas Publicas e Gestao Governamental, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40,  1o, inciso I, da Constituicao da Republica Federativa do Brasil, n

In [23]:
s1 = "CONCEDER, aposentadoria voluntaria integral, a servidora MARIA FRASSINETTE DE FREITAS\nVASCONCELOS, matricula n.o 82.943-9, no cargo de Agente de Gestao de Residuos Solidos, Classe\nUnica, Padrao X, do Quadro de Pessoal do Servico de Limpeza Urbana do Distrito Federal, nos termos\ndo artigo 3o, incisos I, II e III, e Paragrafo unico da Emenda Constitucional n.o 47 de 05/07/2005,"], [56.69187545776367, 80.32613372802734, 404.1012878417969, 874.0826416015625, "combinado com o artigo 44 da Lei Complementar n.o 769, de 30/06/2008. Lotacao: Servico de Limpeza\nUrbana do Distrito Federal. Processo SEI n.o 00094-00012948/2018-15."
s2 = df.loc[15, 'text']

In [24]:
print(s1)

CONCEDER, aposentadoria voluntaria integral, a servidora MARIA FRASSINETTE DE FREITAS VASCONCELOS, matricula n.o 82.943-9, no cargo de Agente de Gestao de Residuos Solidos, Classe Unica, Padrao X, do Quadro de Pessoal do Servico de Limpeza Urbana do Distrito Federal, nos termos do artigo 3o, incisos I, II e III, e Paragrafo unico da Emenda Constitucional n.o 47 de 05/07/2005, combinado com o artigo 44 da Lei Complementar n.o 769, de 30/06/2008. Lotacao: Servico de Limpeza Urbana do Distrito Federal. Processo SEI n.o 00094-00012948/2018-15


In [25]:
print(s2)

CONCEDER, aposentadoria voluntaria integral, a servidora MARIA FRASSINETTE DE FREITAS VASCONCELOS, matricula n.o 82.943-9, no cargo de Agente de Gestao de Residuos Solidos, Classe Unica, Padrao X, do Quadro de Pessoal do Servico de Limpeza Urbana do Distrito Federal, nos termos do artigo 3o, incisos I, II e III, e Paragrafo unico da Emenda Constitucional n.o 47 de 05/07/2005, combinado com o artigo 44 da Lei Complementar n.o 769, de 30/06/2008. Lotacao: Servico de Limpeza Urbana do Distrito Federal. Processo SEI n.o 00094-00012948/2018-15


## Parte 3 - Salvando os dados rotulados padrao IOB